In [ ]:
#load libraries
library(tidyverse)

In [ ]:
#define helper functions
find_library_prep <- function(entry) {
  sapply(entry, function(x) {
    if (grepl("(N)", as.character(x), fixed = TRUE)) {
      "Nextera"
    } else if (grepl("(LA)", as.character(x), fixed = TRUE)) {
      "Linker amplification"
    } else {
      "Standard"
    }
  })
}

convert_sci_to_label <- function(sci_num) {
  format(sci_num, scientific = FALSE)
}

In [ ]:
#load data
article_data <- read_csv("article_data.csv") %>%
  mutate(prep = find_library_prep(`Library amplification (# cycles)`)) %>%
  within({
  `Library amplification (# cycles)` <- as.numeric(str_remove_all(`Library amplification (# cycles)`, "\\(.*\\)"))
  `DNA source` <- str_remove_all(`DNA source`, "Tara Oceans ")
  })

In [ ]:
#plot experiment 1
ggplot(data = filter(article_data, Experiment == "Experiment 1"), aes(x = `Starting DNA (ng)`, y = `Library amplification (# cycles)`)) +
  geom_point(aes(shape = Technology), size = 3) +
  scale_x_continuous(limits = c(0.001, 10000), labels = convert_sci_to_label, trans = "log10") +
  geom_text(aes(label = Replicates), hjust = -1, vjust = -1) +
  labs(x = "Starting DNA amount (ng)",
       y = "Library amplification cycles",
       shape = "Sequencing technology") +
  ggtitle("Experiment 1") +
  theme_bw(base_size = 14) +
  theme(plot.title = element_text(size = 18, face = "bold"),
        panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(),
        panel.background = element_blank(), 
        axis.line = element_line(colour = "black"))

In [ ]:
#plot experiment 2
ggplot(data = filter(article_data, Experiment == "Experiment 2"), aes(x = `Starting DNA (ng)`, y = `Library amplification (# cycles)`)) +
  geom_point(aes(color = prep), shape = "square", size = 3) +
  geom_text(aes(label = Replicates), hjust = -1, vjust = -1) +
  scale_y_continuous(limits = c(0, 25)) +
  scale_x_continuous(limits = c(1, 1000), labels = convert_sci_to_label, trans = "log10") +
  facet_wrap(~`DNA source`) +
  labs(x = "Starting DNA amount (ng)",
       y = "Library amplification cycles",
       color = "Library Prep.") +
  ggtitle("Experiment 2") +
  theme_bw(base_size = 14) +
  theme(plot.title = element_text(size = 18, face = "bold"),
        panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(),
        panel.background = element_blank(), 
        axis.line = element_line(colour = "black"))
